In [5]:
# jupyter notebook --NotebookApp.kernel_spec_manager_class='environment_kernels.EnvironmentKernelSpecManager'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [6]:
# data generating script
def place_cube(matrix, origin, r):
	#dim = matrix.shape()
	matrix[
	origin[0]-r:origin[0]+r,
	origin[1]-r:origin[1]+r,
	origin[2]-r:origin[2]+r,
	] = 1
	return matrix

def dist(x, y):
	return math.sqrt(
		(x[0]-y[0])**2+
		(x[1]-y[1])**2+
		(x[2]-y[2])**2
		)

def place_sphere(matrix, origin, r):
	for index, x in np.ndenumerate(matrix):
	    if dist(index, origin) <= r:
	    	matrix[index] = 1
	return matrix

def random_shapes(m_size,num):
    shapes = np.zeros((m_size,m_size,m_size,num))
    shape_types = np.random.randint(low = 0,high = 2,size= num)
    
    for i in range(num):
        r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
        org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
        if shape_types[i] == 0:
            shapes[:,:,:,i] = place_cube(shapes[:,:,:,i],org,r)
        elif shape_types[i] == 1:
            shapes[:,:,:,i] = place_sphere(shapes[:,:,:,i],org,r)
            
    return shapes, shape_types
        

def matching_shapes(m_size, num):
	cubes = np.zeros((m_size,m_size,m_size,num))
	spheres = np.zeros((m_size,m_size,m_size,num))
	for i in range(num):
		r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
		org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
		cubes[:,:,:,i] = place_cube(cubes[:,:,:,i],org,r)
		spheres[:,:,:,i] = place_sphere(spheres[:,:,:,i],org,r)
	return cubes, spheres

In [7]:
def cnn_model_fn(features, labels, mode):
  #"""Model function for CNN."""
  # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
        

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

  # Generate Predictions
    predictions = {
        "classes": tf.argmax(
          input=logits, axis=1),
        "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

  # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)


In [12]:
def main(unused_argv):
    # Load training and eval data
    mnist = learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    mnist_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

        # Train the model
    mnist_classifier.fit(
        x=train_data,
        y=train_labels,
        batch_size=100,
        steps=2000,
        monitors=[logging_hook])
    
        # Configure the accuracy metric for evaluation
    metrics = {
        "accuracy":
            learn.MetricSpec(
                metric_fn=tf.metrics.accuracy, prediction_key="classes"),
    }
    
    # Evaluate the model and print results
    eval_results = mnist_classifier.evaluate(
        x=eval_data, y=eval_labels, metrics=metrics)
    print(eval_results)



    
    
    


In [11]:
eval_results = mnist_classifier.evaluate(
        x=eval_data, y=eval_labels, metrics=metrics)
print(eval_results)


NameError: name 'mnist_classifier' is not defined

In [9]:
tf.app.run()


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc81b18e470>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_

INFO:tensorflow:probabilities = [[ 0.11700494  0.09312608  0.10864026  0.09090897  0.09781609  0.0907943
   0.10165785  0.09186688  0.10540575  0.10277887]
 [ 0.09793661  0.10127503  0.10209028  0.09655477  0.10383975  0.09542609
   0.09540123  0.11354886  0.1016424   0.09228498]
 [ 0.09572146  0.0936365   0.10053017  0.09484934  0.1011657   0.09356625
   0.1021516   0.10296152  0.11166769  0.10374971]
 [ 0.10506972  0.09662238  0.1158524   0.1000914   0.09407693  0.08677355
   0.09526367  0.10651342  0.10621409  0.0935225 ]
 [ 0.09629111  0.09792481  0.10713397  0.09702091  0.09447802  0.09876429
   0.10469118  0.09511533  0.10987318  0.09870712]
 [ 0.10277196  0.09972847  0.10337717  0.10285465  0.10193867  0.09599607
   0.08356696  0.11322729  0.10056514  0.09597358]
 [ 0.09543246  0.09703803  0.11291189  0.08613496  0.09353481  0.0943849
   0.08697585  0.10780377  0.12158415  0.10419918]
 [ 0.0951477   0.08830739  0.10624263  0.10296617  0.09766721  0.08945002
   0.10727755  0.1087

INFO:tensorflow:global_step/sec: 4.70736
INFO:tensorflow:loss = 2.28861, step = 101 (21.245 sec)
INFO:tensorflow:probabilities = [[ 0.10991646  0.08947998  0.10603006  0.10009818  0.09605806  0.08562548
   0.09593157  0.10865998  0.11206046  0.09613971]
 [ 0.10840856  0.09738999  0.1108143   0.08818866  0.08674234  0.09749278
   0.09510578  0.09443774  0.11661039  0.10480941]
 [ 0.09930397  0.09198856  0.10347943  0.10371062  0.09888512  0.09137066
   0.09577113  0.112       0.09954146  0.10394911]
 [ 0.10194169  0.09691036  0.10425431  0.09529156  0.08425908  0.10265076
   0.09812351  0.10558214  0.10799596  0.1029906 ]
 [ 0.1060086   0.09468669  0.1058528   0.08833789  0.09037545  0.08928896
   0.09775743  0.11059222  0.11425469  0.10284522]
 [ 0.09838765  0.10396501  0.1110798   0.09559595  0.09208936  0.09016254
   0.0907707   0.11055572  0.1070521   0.10034119]
 [ 0.10060153  0.09545761  0.10758428  0.09957054  0.08688718  0.09527668
   0.10356     0.10089375  0.11405273  0.096115

INFO:tensorflow:probabilities = [[ 0.09383132  0.0989908   0.09955116  0.10441409  0.09366986  0.09712751
   0.09463519  0.09886648  0.11154563  0.1073679 ]
 [ 0.10227557  0.09530281  0.09717791  0.09550275  0.10194809  0.09339441
   0.09544498  0.10865296  0.10859028  0.10171027]
 [ 0.10460287  0.09130275  0.11734232  0.10192377  0.09110502  0.09181667
   0.09140857  0.10190018  0.11075608  0.09784169]
 [ 0.10009106  0.09830799  0.10739253  0.10774892  0.09276768  0.08699004
   0.09819904  0.10096534  0.10967264  0.09786483]
 [ 0.09575281  0.10484164  0.09374     0.10499854  0.09486556  0.09820531
   0.09373811  0.10763167  0.10047074  0.10575565]
 [ 0.12080708  0.09700464  0.10798713  0.09598239  0.08634076  0.08444965
   0.10596968  0.089846    0.11536971  0.09624292]
 [ 0.104283    0.0941344   0.10497662  0.09293328  0.09954317  0.09642708
   0.1026452   0.09915527  0.10770098  0.09820094]
 [ 0.10038704  0.09466653  0.09997216  0.09953538  0.10585442  0.09228591
   0.10560083  0.09

INFO:tensorflow:global_step/sec: 4.44558
INFO:tensorflow:loss = 2.27985, step = 201 (22.494 sec)
INFO:tensorflow:probabilities = [[ 0.10334715  0.09119681  0.09503855  0.10799387  0.0936655   0.09154247
   0.10107541  0.10786463  0.1067735   0.10150209]
 [ 0.10775076  0.08546248  0.10611477  0.10080566  0.09282788  0.08970401
   0.09391269  0.10567014  0.122231    0.09552056]
 [ 0.10636916  0.09745657  0.1105241   0.0987239   0.08897651  0.10018425
   0.09607287  0.09689484  0.10797041  0.09682736]
 [ 0.10101356  0.10804234  0.10417243  0.09738132  0.09940736  0.09199072
   0.09427401  0.09779581  0.10664148  0.09928105]
 [ 0.10363396  0.10648955  0.11115848  0.09318426  0.08922165  0.09309857
   0.09216806  0.10986049  0.09944472  0.10174026]
 [ 0.10223088  0.09890454  0.09396712  0.10727727  0.09384057  0.09193522
   0.0989513   0.10311303  0.11267857  0.09710147]
 [ 0.112491    0.09843464  0.11474814  0.10147316  0.09434     0.10162863
   0.08627804  0.09738484  0.10402092  0.089200

...

INFO:tensorflow:probabilities = [[ 0.00009459  0.00002971  0.00001526  0.0000385   0.01384707  0.0001032
   0.00000201  0.00963517  0.00073     0.97550452]
 [ 0.00001301  0.00000404  0.00083389  0.99881554  0.00000017  0.00026827
   0.00000006  0.00000596  0.00005485  0.00000421]
 [ 0.00246181  0.20655666  0.41704044  0.10635956  0.01445167  0.0017113
   0.0052629   0.10361546  0.05919138  0.08334885]
 [ 0.00010819  0.00000807  0.0010139   0.00000308  0.92787296  0.00094823
   0.06787965  0.00002206  0.00022121  0.00192266]
 [ 0.00879238  0.00000402  0.00139857  0.00006145  0.00011942  0.00398151
   0.9855386   0.00000054  0.00008617  0.00001735]
 [ 0.00005848  0.00073147  0.0005487   0.00067671  0.979451    0.00131236
   0.00556674  0.0017202   0.00105449  0.00887986]
 [ 0.9971469   0.00000006  0.00055422  0.00006871  0.00000032  0.00058686
   0.00162236  0.00000162  0.0000156   0.00000327]
 [ 0.00014138  0.00017615  0.03087749  0.9534418   0.00000535  0.00138532
   0.00001824  0.0000

INFO:tensorflow:global_step/sec: 3.9253
INFO:tensorflow:loss = 0.332949, step = 5601 (25.476 sec)
INFO:tensorflow:probabilities = [[ 0.00012012  0.00040388  0.00003684  0.00006452  0.1872292   0.00182334
   0.00005851  0.01242612  0.00495334  0.79288417]
 [ 0.00052162  0.96888936  0.00679608  0.00367961  0.0016838   0.00112999
   0.00572884  0.00465165  0.00397735  0.00294174]
 [ 0.00019769  0.00513081  0.90744072  0.05448511  0.00000024  0.00038624
   0.00075661  0.0000076   0.03159378  0.00000117]
 [ 0.01061804  0.00002149  0.28032774  0.00011612  0.0060703   0.01822718
   0.68056917  0.00014512  0.00229914  0.00160566]
 [ 0.00198695  0.00023498  0.00614085  0.00008265  0.00568397  0.00783308
   0.97769219  0.00000455  0.00026996  0.00007073]
 [ 0.99731642  0.          0.00010503  0.00001921  0.          0.00255494
   0.00000353  0.00000021  0.00000067  0.0000001 ]
 [ 0.00163858  0.00000063  0.00085699  0.00342718  0.00615772  0.00019293
   0.00000786  0.13696447  0.00179938  0.84895

INFO:tensorflow:probabilities = [[ 0.00004039  0.00000117  0.00000792  0.00006068  0.00001286  0.0004758
   0.00000004  0.99769622  0.00025856  0.00144634]
 [ 0.00391462  0.00089886  0.90695113  0.00812776  0.00000065  0.00046198
   0.07835796  0.00000016  0.00128562  0.00000114]
 [ 0.00064952  0.0000004   0.00000329  0.00148953  0.00002207  0.99507844
   0.00022463  0.00000013  0.00249539  0.00003661]
 [ 0.00139758  0.00014964  0.01133582  0.00298239  0.00000556  0.00297396
   0.00051776  0.00000335  0.98055202  0.00008195]
 [ 0.00049705  0.00005982  0.00203765  0.0002432   0.00008306  0.00214274
   0.01864643  0.00022669  0.97516656  0.0008969 ]
 [ 0.0295202   0.000016    0.2662794   0.00262772  0.00281324  0.00339021
   0.69182068  0.00071986  0.00069706  0.00211568]
 [ 0.00423153  0.002336    0.26676068  0.49221191  0.0004673   0.00493546
   0.00276751  0.00011742  0.22366723  0.00250499]
 [ 0.0016714   0.92250681  0.00348459  0.00185994  0.00133852  0.00304655
   0.02985318  0.000

INFO:tensorflow:global_step/sec: 4.04503
INFO:tensorflow:loss = 0.272628, step = 5701 (24.722 sec)
INFO:tensorflow:probabilities = [[ 0.01090035  0.00000455  0.00333033  0.90579605  0.00015073  0.06510282
   0.00007105  0.0000949   0.01264617  0.0019031 ]
 [ 0.00001732  0.00000161  0.00006702  0.00379896  0.00000015  0.00002441
   0.00000001  0.99549979  0.00003152  0.00055925]
 [ 0.00807673  0.0002788   0.03256841  0.93041039  0.0000015   0.01615695
   0.00022473  0.00093178  0.01127804  0.00007258]
 [ 0.00010607  0.00081752  0.00010948  0.00005896  0.00019395  0.0001127
   0.00000159  0.98474729  0.00059068  0.01326183]
 [ 0.0044283   0.04411672  0.01065426  0.00461616  0.01183741  0.03411694
   0.88355601  0.00014807  0.00608574  0.00044038]
 [ 0.00092445  0.00000148  0.00853399  0.02286793  0.00068211  0.73259395
   0.0046419   0.00000958  0.22553444  0.0042102 ]
 [ 0.04847934  0.00000043  0.0025693   0.00033457  0.00203763  0.00102721
   0.00185727  0.32815111  0.00037709  0.61516

KeyboardInterrupt: 